In [1]:
import numpy as np
import torch
import sys
from utils_old import *
from matplotlib import pyplot as plt
import torch.nn as nn
import time
from numba import cuda
from tqdm import tqdm, trange
import os
import pyroc
import pandas as pd
import gc
from IPython.display import clear_output
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"  
device = torch.device("cuda:0")
dtype = torch.float32
torch.manual_seed(42)
np.random.seed(42)

/math/home/eruisun/software/anaconda/envs/LFI/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

H = 500
out= 100
L = 1
class DN(torch.nn.Module):
    def __init__(self, H=300, out=100):
        super(DN, self).__init__()
        self.restored = False
        self.model = torch.nn.Sequential(
            torch.nn.Linear(28, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, out, bias=True),
        )
    def forward(self, input):
        output = self.model(input)
        return output

class another_DN(torch.nn.Module):
    def __init__(self, H=300, out=100):
        super(another_DN, self).__init__()
        self.restored = False
        self.model = torch.nn.Sequential(
            torch.nn.Linear(28, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, 28, bias=True),
        )
    def forward(self, input):
        output = self.model(input) + input
        return output

class Classifier(torch.nn.Module):
    def __init__(self, H=300, layers = 5):
        super(Classifier, self).__init__()
        self.restored = False
        self.model = torch.nn.Sequential(
            torch.nn.Linear(28, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, 1, bias=True),
            torch.nn.Sigmoid(),
        )
        if layers == 6:
            self.model = torch.nn.Sequential(
                torch.nn.Linear(28, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, 1, bias=True),
                torch.nn.Sigmoid(),
            )
    def forward(self, input):
        output = self.model(input)
        return output

In [3]:
dataset = np.load('HIGGS.npy')
dataset_P = dataset[dataset[:,0]==0][:, 1:] # background (5829122, 28)
dataset_Q = dataset[dataset[:,0]==1][:, 1:] # signal     (5170877, 28)
del dataset

In [4]:
n_train = 1300101
path ='./checkpoint1300101/25/'
model = DN(300, 100).cuda()
another_model = another_DN(300, 100).cuda()
model,another_model,epsilonOPT,sigmaOPT,sigma0OPT,cst = load_model(model, another_model, path)

In [5]:
# 看(n_eval,m)
# 选定 X_eval, Y_eval, 还要算phi=Eq[k(y,z)]-Ep[k(x,z)]的threhold在哪里
# 必须手算!
def generate_PQ(n_eval, Samples, batch_size = 10000, If_n_large_MonteCarlo = 1000):
    X_eval = dataset_P[np.random.choice(dataset_P.shape[0], n_eval, replace=False)]
    Y_eval = dataset_Q[np.random.choice(dataset_Q.shape[0], n_eval, replace=False)]
    EKxx, EKyy, EKxy = compute_gamma(X_eval, Y_eval, model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst, MonteCarlo=If_n_large_MonteCarlo)
    # 算定下来X_eval, Y_eval之后phi的真实的分布
    batches = (Samples-1)//batch_size + 1
    P_scores = np.zeros(Samples)
    Q_scores = np.zeros(Samples)
    for i in trange(batches):
        remain = batch_size
        if i==batches-1:
            remain = Samples - batch_size*(batches-1)
        S_hand = np.concatenate((dataset_P[np.random.choice(dataset_P.shape[0], remain, replace=False)],
                    dataset_Q[np.random.choice(dataset_Q.shape[0], remain, replace=False)]), axis=0)
        S_hand = MatConvert(S_hand, device, dtype)
        PQhat_hand = compute_score_func(S_hand, X_eval, Y_eval, 
                    model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst,
                    M = n_eval)
        PQhat_hand = PQhat_hand.cpu().detach().numpy()
        P_scores[i*batch_size: i*batch_size+remain] = PQhat_hand[:remain]
        Q_scores[i*batch_size: i*batch_size+remain] = PQhat_hand[remain:]
    return X_eval, Y_eval, P_scores, Q_scores, EKxx, EKyy, EKxy


In [6]:
# 注意P_scores, Q_scores已经考虑到n_eval贡献的方差
# EKxx, EKyy, EKxy完全是n_eval决定
class PQ_data():
    def __init__(self, P_scores, Q_scores, EKxx, EKyy, EKxy):
        self.P_scores = P_scores
        self.Q_scores = Q_scores
        self.P_mean = np.mean(P_scores)
        self.P_std = np.std(P_scores)
        self.Q_mean = np.mean(Q_scores)
        self.Q_std = np.std(Q_scores)
        self.EKxx = EKxx
        self.EKyy = EKyy
        self.EKxy = EKxy
    def pval_T_m_in_sigma(self, pi, m, use_gaussian, MonteCarlo):
        T = pi*self.Q_mean + (1-pi)*self.P_mean
        # use_gaussian: 是否用高斯近似, 0是用MonteCarlo近似，1是用高斯近似
        P_scores = self.P_scores
        mean = self.P_mean
        std = self.P_std
        if m==1:
            p = np.mean(P_scores > T)
            p = -scipy.stats.norm.ppf(p)
        if use_gaussian:
            p = (T-mean)/std*np.sqrt(m)
        else:
            T_mix_MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                idx = np.random.choice(P_scores.shape[0], m, replace=False)
                T_mix_MonteCarlo_list[i] = np.mean(P_scores[idx])
            p = np.mean(T_mix_MonteCarlo_list > T)
            p = -scipy.stats.norm.ppf(p)
        self.p = p
        return p
    def type_1_error_H0(self, pi, m, use_gaussian, MonteCarlo):
        P_scores = self.P_scores
        Q_scores = self.Q_scores
        mean = self.P_mean
        std = self.P_std
        gamma = self.EKxx*(pi/2-1) + self.EKxy*(1-pi) + self.EKyy*(pi/2)
        self.gamma = gamma
        if m==1:
            type_1_error = np.mean(P_scores > gamma)
        if use_gaussian:
            type_1_error = 1-scipy.stats.norm.cdf((gamma-mean)/std*np.sqrt(m))
        else:
            MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                idx = np.random.choice(P_scores.shape[0], m, replace=False)
                MonteCarlo_list[i] = np.mean(P_scores[idx])
            type_1_error = np.mean(MonteCarlo_list > gamma)
        self.type_1_error = type_1_error
        return type_1_error
    def type_2_error_H1(self, pi, m, use_gaussian, MonteCarlo):
        P_scores = self.P_scores
        Q_scores = self.Q_scores
        P_mean = self.P_mean
        P_std = self.P_std
        Q_mean = self.Q_mean
        Q_std = self.Q_std
        gamma = self.EKxx*(pi/2-1) + self.EKxy*(1-pi) + self.EKyy*(pi/2)
        self.gamma = gamma
        if m==1:
            type_2_error = np.mean(Q_scores < gamma)
        if use_gaussian:
            mean = Q_mean*pi + P_mean*(1-pi)
            std = np.sqrt(pi*Q_std**2 + (1-pi)*P_std**2)
            type_2_error = scipy.stats.norm.cdf((gamma-mean)/std*np.sqrt(m))
        else:
            MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                Signals_idx = np.random.choice(Q_scores.shape[0], int(m*pi), replace=False)
                Backgrounds_idx = np.random.choice(P_scores.shape[0], int(m*(1-pi)), replace=False)
                MonteCarlo_list[i] = np.mean(np.concatenate((Q_scores[Signals_idx], P_scores[Backgrounds_idx])))
            type_2_error = np.mean(MonteCarlo_list < gamma)
        self.type_2_error = type_2_error
        return type_2_error

In [7]:
torch.cuda.empty_cache()
gc.collect()

9

# pi=0.1

In [36]:
pi= 0.1
n_list = np.array([2, 6, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000,])
m_list = np.array([1, 2, 6, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000,])

n_list = 10**np.linspace(0.5, 4.3, 40)
n_list = n_list.astype(int)
m_list = 10**np.linspace(0.5, 4.3, 40)
m_list = m_list.astype(int)

repeat_sample_n_eval = 2000
Pval_mat = np.zeros((len(n_list), len(m_list),repeat_sample_n_eval))
Type1_mat = np.zeros((len(n_list), len(m_list),repeat_sample_n_eval))
Type2_mat = np.zeros((len(n_list), len(m_list),repeat_sample_n_eval))
Use_Gaussian = 1

In [ ]:
for r in range(repeat_sample_n_eval):
    for i in trange(len(n_list)):
        with torch.no_grad():
            torch.cuda.empty_cache()
            n_eval = n_list[i]
            X_eval, Y_eval, P_scores, Q_scores, EKxx, EKyy, EKxy = generate_PQ(n_eval, Samples=20000, batch_size=4000, If_n_large_MonteCarlo=10)
            My_class = PQ_data(P_scores, Q_scores, EKxx, EKyy, EKxy)
            for j in range(len(m_list)):
                m = m_list[j]
                Pval_mat[i,j,r] += My_class.pval_T_m_in_sigma(pi, m, use_gaussian=Use_Gaussian, MonteCarlo=0)
                Type1_mat[i,j,r] += My_class.type_1_error_H0(pi, m, use_gaussian=Use_Gaussian, MonteCarlo=0)
                Type2_mat[i,j,r] += My_class.type_2_error_H1(pi, m, use_gaussian=Use_Gaussian, MonteCarlo=0)
            del My_class, X_eval, Y_eval, P_scores, Q_scores, EKxx, EKyy, EKxy
            gc.collect()
        clear_output()   
        print(r)



In [ ]:
plt.figure(figsize=(20,4))
# plt.subplot(1,4,1)
# plt.contourf(np.log10(n_list), np.log10(m_list), Pval_mat.T, levels=20)
# plt.colorbar()
# plt.xlabel('$lg(n_{eval})$')
# plt.ylabel('lg(m)')
# plt.title('p-value')

plt.subplot(1,4,2)
plt.contourf(np.log10(n_list), np.log10(m_list), Type1_mat.T, levels=20)
plt.colorbar()
plt.xlabel('$lg(n_{eval})$')
plt.ylabel('lg(m)')
plt.title('$P_{H_0}(fail)$')

plt.subplot(1,4,3)
plt.contourf(np.log10(n_list), np.log10(m_list), Type2_mat.T, levels=20)
plt.colorbar()
plt.xlabel('$lg(n_{eval})$')
plt.ylabel('lg(m)')
plt.title('$P_{H_1}(fail)$')

plt.subplot(1,4,4)
plt.contourf(np.log10(n_list), np.log10(m_list), Type1_mat.T+Type2_mat.T, levels=20)
plt.colorbar()
plt.xlabel('lg(n_eval)')
plt.ylabel('lg(m)')
plt.title('$\Sigma P(fail)$')

plt.suptitle(r'Fix kernel ($n_{train}=1.3e6$), Use Gaussian Approx='+str(Use_Gaussian)+', $\pi$='+str(pi))

In [ ]:
type123 = Type1_mat+Type2_mat
plt.matshow(type123)